In [1]:
import numpy as np
from numpy import dot 
import pandas as pd 

from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer)
from sklearn.decomposition import TruncatedSVD
import scipy

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
corpus  =  {"I am royalty",
            "I have destiny",
            "I have been set free, I am truly free",
            "I'm gonna shape history"
           }

In [3]:
corpus

{'I am royalty',
 'I have been set free, I am truly free',
 'I have destiny',
 "I'm gonna shape history"}

## Term Document Matrix with Frequencies of Words 

In [4]:
bag_of_words = CountVectorizer().fit_transform(corpus)
term_document_mat = bag_of_words.todense()
print(term_document_mat)
term_document_mat.shape

[[1 0 0 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 1 0 0 0 0 0]
 [1 1 0 2 0 1 0 0 1 0 1]
 [0 0 0 0 1 0 1 0 0 1 0]]


(4, 11)

- The rows of the term-document matrix are the documents or sentences and the columns are the words or terms  
- We can view the rows as document vectors whose entries are frequencies of words 

## Term Document Matrix with TF-IDF

In [5]:
vectorizer1= TfidfVectorizer(min_df=1, stop_words="english")
bag_of_words1 = vectorizer1.fit_transform(corpus)

term_document_mat1 = bag_of_words1.todense()
term_document_mat1 = np.round(term_document_mat1, 4)
print(term_document_mat1)
#term_document_mat1.shape

[[0.     0.     0.     0.     1.     0.     0.     0.    ]
 [1.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.8165 0.     0.     0.     0.4082 0.     0.4082]
 [0.     0.     0.5774 0.5774 0.     0.     0.5774 0.    ]]


## Terms Used in the Term-Document Matrix 

In [6]:
terms = vectorizer1.get_feature_names()
print(terms) 

['destiny', 'free', 'gonna', 'history', 'royalty', 'set', 'shape', 'truly']


## Term Document Matrix in a DataFrame

In [7]:
pd.DataFrame(term_document_mat1, 
             columns=terms,
            index = ["d1", "d2", "d3", "d4"])

,destiny,free,gonna,history,royalty,set,shape,truly
d1,0.0,0.0000,0.0000,0.0000,1.0,0.0000,0.0000,0.0000
d2,1.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000
d3,0.0,0.8165,0.0000,0.0000,0.0,0.4082,0.0000,0.4082
d4,0.0,0.0000,0.5774,0.5774,0.0,0.0000,0.5774,0.0000


## Implement Latent Semantic Analysis


In [8]:
# decompose the term-document matrix using singular value decomposition 

In [9]:
svd = TruncatedSVD(n_components=2)
lsa = svd.fit_transform(term_document_mat1)
lsa

array([[ 0.00000000e+00,  1.00000000e+00],
       [-1.11686954e-12,  0.00000000e+00],
       [-1.84446135e-12,  0.00000000e+00],
       [ 1.00008614e+00,  0.00000000e+00]])

- The implementation of the truncated svd returns a reduced or lower rank approximation of the term-document matrix. The output is the matrix multiplication of the left singular matrix and the diagnal matrix that were obtained from svd. 
- This output is a document-topic matrix where the rows are document vectors whose entries indicate the extends to which a document captures a topic. 

## How is the Reduced Rank Matrix Obtained?

In [10]:
# The direct implementation of svd in scipy 
U, s, VT =scipy.linalg.svd(term_document_mat1)
print(U)

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]


In [11]:
print(s)

[1.00008614 1.         1.         0.99996336]


In [12]:
VT = np.round(VT, 2)
VT

array([[ 0.  ,  0.  ,  0.58,  0.58,  0.  ,  0.  ,  0.58,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ],
       [ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.82,  0.  ,  0.  ,  0.  ,  0.41,  0.  ,  0.41],
       [ 0.  ,  0.33, -0.64,  0.47,  0.  , -0.33,  0.17, -0.33],
       [ 0.  , -0.33, -0.32,  0.24,  0.  ,  0.83,  0.09, -0.17],
       [ 0.  ,  0.  , -0.21, -0.58,  0.  , -0.  ,  0.79, -0.  ],
       [ 0.  , -0.33, -0.32,  0.24,  0.  , -0.17,  0.09,  0.83]])

### Approximating the term-document matrix 
- If the term-document matrix has mxn dimensions, where m = number of rows (documents) and n = number of columns (words or terms) then: 
- $ A = U * Sigma * VT$
- If we reduce the number of singular values to k, then sigma becomes sigma_k, U and VT will also be reduced to U_k and VT_k respectively. 

- Then, we can approximate k as: $A_k = U_k*Sigma_k*VT_k$

- Where $A_K$ is mxn, $U_K$ is mxk and $VT_k$ is kxn and sigma is kxk matrix
- The reduced matrix is $U_k*Sigma_k$ and the encoding matrix is $Sigma_k*VT_k$ 
- Note that in the literature, the term-document matrix is represented such that the columns are documents and the rows are words columns. This affects the math of how we get the reduced matrix and the encoding matrix. If we want to implement this in code, we will need to swap the formulas for encoding and reduced matrix specified above, because the term-document matrix in this situation is the transpose of the term document matrix we specified above. 

## Implementing a Low Rank Approximation 
- Let's set k=2

In [13]:
U2 = U[:, 0:2]
U2

array([[0., 1.],
       [0., 0.],
       [0., 0.],
       [1., 0.]])

In [14]:
sigma = np.diag(s)
sigma

array([[1.00008614, 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.99996336]])

In [15]:
sigma2 =sigma[0:2:,0:2]
sigma2

array([[1.00008614, 0.        ],
       [0.        , 1.        ]])

In [16]:
VT2 = VT[0:2]
VT2

array([[0.  , 0.  , 0.58, 0.58, 0.  , 0.  , 0.58, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ]])

In [17]:
# document-topic matrix - documents are rows 

reduced_rank_matrix = np.matmul(U2, sigma2) # same as U2.dot(sigma2)
reduced_rank_matrix

array([[0.        , 1.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [1.00008614, 0.        ]])

- This reduced matrix obtained by multipy U2 by sigma2 is the same as the truncated svd obtained from the sklearn package. 
- **The rows of the reduced rank matrix are document vectors in the reduced space**

In [18]:
# truncated svd returnd from the sklearn package
np.round(lsa, 2)

array([[ 0.,  1.],
       [-0.,  0.],
       [-0.,  0.],
       [ 1.,  0.]])

## The Encoding Matrix
- The encoding matrix is the obtained from multiplying (scaling) matrix (VT) by the diagonal matrix: $Sigma_k*VTk$
- The rows of the encoding matrix are word vectors whose entries indicate the extend to which the word contributes in forming a topic. 
- we can obtain this matrix using the svd object that we created using the trucated svd in sklearn as follows:


In [19]:
np.round(svd.components_.T, 2)

array([[-0.  ,  0.  ],
       [-0.  ,  0.  ],
       [ 0.58, -0.  ],
       [ 0.58, -0.  ],
       [-0.  ,  1.  ],
       [-0.  , -0.  ],
       [ 0.58, -0.  ],
       [-0.  , -0.  ]])

- We can also obtain the encoding matrix using matrix multiplcation of sigma2 and VT2
- The matrix should be the similar to that obtained from sklearn
- The matrix multiplication of sigma2 and VT2 can be obtained as follows
- **The rows of the enconding matrix are word vectors in the reduced space**

In [20]:
encoding_mat = np.round(np.matmul(sigma2, VT2).T, 2)
encoding_mat

array([[0.  , 0.  ],
       [0.  , 0.  ],
       [0.58, 0.  ],
       [0.58, 0.  ],
       [0.  , 1.  ],
       [0.  , 0.  ],
       [0.58, 0.  ],
       [0.  , 0.  ]])

In [21]:
encoding_df = pd.DataFrame(encoding_mat, 
                           columns=["topic_1", "topic_2"],
                           index = terms)
encoding_df

,topic_1,topic_2
destiny,0.00,0.0
free,0.00,0.0
gonna,0.58,0.0
history,0.58,0.0
royalty,0.00,1.0
set,0.00,0.0
shape,0.58,0.0
truly,0.00,0.0


In [22]:
reduced_data = pd.DataFrame(reduced_rank_matrix, 
                            columns=["topic_1", "topic_2"])
reduced_data

,topic_1,topic_2
0,0.000000,1.0
1,0.000000,0.0
2,0.000000,0.0
3,1.000086,0.0


In [23]:
reduced_data["document"] = corpus
# reorder columns 
reduced_data = reduced_data[["document", "topic_1", "topic_2"]] 
reduced_data

,document,topic_1,topic_2
0,I am royalty,0.000000,1.0
1,I have destiny,0.000000,0.0
2,"I have been set free, I am truly free",0.000000,0.0
3,I'm gonna shape history,1.000086,0.0


## Semantic Comparison and Information Retrieval

If there is another document which serves as a query or test data, we can compare that document with the document with the training data or corpus and retrieve the most similar document to the query. For example, let our query be as follows:

In [24]:
query = "Those who are royalty shape their generation"

- We need to represent this document in the reduced space using the centriod of the word vectors for words in the query document. 

- Words in the query that are in the set of terms used for the term-document matrix are as follows:

In [25]:
query_terms = [word for word in terms if word in query.split()]
query_terms

['royalty', 'shape']

In [26]:
# find the word vectors of these words from the encoding matrix 
word_vectors = encoding_df.loc[query_terms]
word_vectors 

,topic_1,topic_2
royalty,0.00,1.0
shape,0.58,0.0


In [27]:
word_vectors = np.array(word_vectors)
word_vectors 

array([[0.  , 1.  ],
       [0.58, 0.  ]])

In [28]:
# find mean across the rows 
query_document= np.mean(word_vectors, axis=0)
query_document

array([0.29, 0.5 ])

- The cosine similarity of query and another document in the training set is the dot product of the query document and the other document, divided by the the product of the lengths of the document vectors   
- K(X, Y) = <X, Y> / (||X||*||Y||)
- let's find semantic similarity between query and the first document in the reduced space. 

In [29]:
# the first document in the reduced space 
cosine_similarity(query_document.reshape(1, -1), 
                  reduced_rank_matrix[0].reshape(1, -1))

array([[0.86503119]])

In [30]:
# similarity of query with all the documents
cosine_sim = cosine_similarity(query_document.reshape(1, -1),
                               reduced_rank_matrix)
cosine_sim


array([[0.86503119, 0.        , 0.        , 0.50171809]])

In [31]:
reduced_data

,document,topic_1,topic_2
0,I am royalty,0.000000,1.0
1,I have destiny,0.000000,0.0
2,"I have been set free, I am truly free",0.000000,0.0
3,I'm gonna shape history,1.000086,0.0


In [32]:
# add cosine similarity to the data frame 
sim = cosine_sim.tolist()[0]
reduced_data = reduced_data.assign(cosine_sim=sim)
reduced_data

,document,topic_1,topic_2,cosine_sim
0,I am royalty,0.000000,1.0,0.865031
1,I have destiny,0.000000,0.0,0.000000
2,"I have been set free, I am truly free",0.000000,0.0,0.000000
3,I'm gonna shape history,1.000086,0.0,0.501718


In [33]:
# Sort the documents by their cosine similarity
reduced_data = reduced_data.sort_values(by="cosine_sim", ascending=False)
reduced_data

,document,topic_1,topic_2,cosine_sim
0,I am royalty,0.000000,1.0,0.865031
3,I'm gonna shape history,1.000086,0.0,0.501718
1,I have destiny,0.000000,0.0,0.000000
2,"I have been set free, I am truly free",0.000000,0.0,0.000000


- Retrieve the document in the training set or corpus that is most similar to the query document 


In [34]:
reduced_data["document"].iloc[0]

'I am royalty'

- The next similar document is:

In [35]:
reduced_data["document"].iloc[1]

"I'm gonna shape history"

In [36]:
# check query document again 
query

'Those who are royalty shape their generation'